In [ ]:
#上証指數
import akshare as ak

#shenzhen

stock_szse_summary_df = ak.stock_szse_summary(date="20250221")
print(stock_szse_summary_df)


#shanghai 
stock_sse_summary_df = ak.stock_szse_summary(date="20250221")
print(stock_sse_summary_df)

      证券类别     数量          成交金额           总市值          流通市值
0       股票   2892  1.337843e+12  3.543463e+13  3.011052e+13
1     主板A股   1484  6.680070e+11  2.149945e+13  1.942046e+13
2     主板B股     39  1.400922e+08  5.132889e+10  5.119274e+10
3    创业板A股   1369  6.696957e+11  1.388385e+13  1.063886e+13
4       基金    754  8.151741e+10  1.154960e+12  1.121386e+12
5      ETF    442  8.040393e+10  1.060125e+12  1.060125e+12
6      LOF    290  8.871437e+08  3.546362e+10  3.546362e+10
7    封闭式基金      1  5.063119e+06  1.679609e+09  1.679609e+09
8   基础设施基金     21  2.212723e+08  5.769171e+10  2.411857e+10
9       债券  15152  3.249646e+11           NaN           NaN
10    债券现券  14493  6.917059e+10  8.413431e+13  2.903846e+12
11    债券回购     27  2.554440e+11           NaN           NaN
12     ABS    632  3.500490e+08  4.372143e+11  4.372143e+11
13      期权    608  1.169385e+09           NaN           NaN
      证券类别     数量          成交金额           总市值          流通市值
0       股票   2892  1.337843e+12  3.54346

In [ ]:
import akshare as ak
import pandas as pd
# 获取东方财富网的概念板块数据

industry_board_df = ak.stock_board_industry_name_em()

industry_board_df.to_csv(r"C:\Quant\quant_HC\data\stocks\industry_board_df.csv", index=False)
print(industry_board_df)


In [105]:
import akshare as ak

# 检查 akshare 版本（需 >= 1.10.0）
print("akshare 版本:", ak.__version__)

# 获取概念板块数据
try:
    df = ak.stock_board_concept_name_em()
    print("前5行数据示例:\n", df.head())
    print("\n列名检查:", df.columns.tolist())
    
    # 检查关键字段是否有有效数据
    if df["涨跌幅"].sum() == 0:
        print("警告：涨跌幅全为0，可能未在交易时段运行！")
except Exception as e:
    print("接口调用失败，错误信息:", e)

akshare 版本: 1.16.10
前5行数据示例:
    排名        板块名称    板块代码      最新价  涨跌额  涨跌幅            总市值  换手率  上涨家数  下跌家数  \
0   1         腾讯云  BK1189  1082.97  0.0  0.0   728285904000  0.0     0     0   
1   2  DeepSeek概念  BK1188  1367.65  0.0  0.0  3374162160000  0.0     0     0   
2   3       小红书概念  BK1187  1141.88  0.0  0.0   495616336000  0.0     0     0   
3   4        首发经济  BK1186   968.55  0.0  0.0   318992288000  0.0     0     0   
4   5        冰雪经济  BK1185   878.81  0.0  0.0   227548616000  0.0     0     0   

  领涨股票  领涨股票-涨跌幅  
0    -       NaN  
1    -       NaN  
2    -       NaN  
3    -       NaN  
4    -       NaN  

列名检查: ['排名', '板块名称', '板块代码', '最新价', '涨跌额', '涨跌幅', '总市值', '换手率', '上涨家数', '下跌家数', '领涨股票', '领涨股票-涨跌幅']
警告：涨跌幅全为0，可能未在交易时段运行！


In [81]:
import akshare as ak

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
print(stock_zh_a_spot_em_df)


        序号      代码    名称     最新价    涨跌幅    涨跌额       成交量           成交额     振幅  \
0        1  301003  江苏博云   28.14  20.00   4.69   28112.0  7.675015e+07  16.80   
1        2  300539  横河精密   16.20  20.00   2.70  152870.0  2.286527e+08  20.00   
2        3  300031  宝通科技   31.45  19.99   5.24  184534.0  5.803599e+08   0.00   
3        4  300214  日科化学    8.32  19.03   1.33  629392.0  5.110682e+08  16.17   
4        5  873527   夜光明   17.80  17.49   2.65   40005.0  6.715532e+07  15.38   
...    ...     ...   ...     ...    ...    ...       ...           ...    ...   
5686  5687  600602  云赛智联   26.98  -6.32  -1.82  854962.0  2.372115e+09   6.01   
5687  5688  600825  新华传媒    7.30  -6.41  -0.50  484299.0  3.567987e+08   4.74   
5688  5689  688692  达梦数据  376.88  -6.48 -26.12    4250.0  1.624194e+08   5.75   
5689  5690  300660  江苏雷利   66.64  -6.63  -4.73  145165.0  9.700837e+08   6.96   
5690  5691  301186  超达装备   47.01  -7.13  -3.61   27183.0  1.300697e+08  11.54   

          最高  ...     量比   

In [74]:
import akshare as ak

# Fetch the stock data
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()

# Save the DataFrame to a CSV file (corrected file path)
stock_zh_a_spot_em_df.to_csv(r"C:\Quant\quant_HC\data\stocks\stock_zh_a_spot_em.csv", index=False)

print("CSV file has been saved as 'stock_zh_a_spot_em.csv' in the specified directory.")


CSV file has been saved as 'stock_zh_a_spot_em.csv' in the specified directory.


In [88]:
# 综合筛选大资金流入候选股
candidates = stock_zh_a_spot_em_df[
    (stock_zh_a_spot_em_df["量比"] > 2) &
    (stock_zh_a_spot_em_df["换手率"] > 15) &
    (stock_zh_a_spot_em_df["涨跌幅"] > 5) &
    (stock_zh_a_spot_em_df["成交额"] > 5e8)
]

print("大资金流入候选股：")
print(candidates[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

大资金流入候选股：
         代码    名称    最新价    涨跌幅    换手率     量比           成交额
3    300214  日科化学   8.32  19.03  15.60  19.31  5.110682e+08
59   300846  首都在线  29.60   7.68  15.90   5.41  1.794553e+09
63   301013   利和兴  18.06   7.50  19.31  12.81  6.537943e+08
71   600203  福日电子  13.03   7.07  22.34  30.64  1.751149e+09
73   600355  精伦电子   4.45   6.97  23.12  27.68  5.091574e+08
79   002765  蓝黛科技  14.42   6.58  15.24  10.66  1.286020e+09
112  002369  卓翼科技  13.24   5.67  26.94  18.38  2.011776e+09


In [96]:
# 综合筛选大资金流入候选股
candidates_2 = stock_zh_a_spot_em_df[
    (stock_zh_a_spot_em_df["涨跌幅"] > 5)
]

print("大资金流入候选股：")
print(candidates_2[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

大资金流入候选股：
         代码    名称    最新价    涨跌幅    换手率     量比           成交额
0    301003  江苏博云  28.14  20.00   3.67  26.95  7.675015e+07
1    300539  横河精密  16.20  20.00   8.91   9.68  2.286527e+08
2    300031  宝通科技  31.45  19.99   5.29   3.76  5.803599e+08
3    300214  日科化学   8.32  19.03  15.60  19.31  5.110682e+08
4    873527   夜光明  17.80  17.49  11.01  26.64  6.715532e+07
..      ...   ...    ...    ...    ...    ...           ...
135  688623  双元科技  59.80   5.08   1.19   6.83  1.303668e+07
136  301396  宏景科技  52.65   5.03  11.85   3.74  3.192754e+08
137  002569  ST步森   9.20   5.02   0.91  12.38  1.148175e+07
138  600608  ST沪科   3.56   5.01   0.64   7.29  7.116564e+06
139  300521   爱司凯  22.64   5.01   1.82   4.90  5.777237e+07

[140 rows x 7 columns]


In [97]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = r"C:\Quant\quant_HC\data\stocks\stock_zh_a_spot_em.csv"
stock_df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
# 综合筛选大资金流入候选股
candidates_1 = stock_df[
    (stock_df["量比"] > 2) &
    (stock_df["换手率"] > 15) &
    (stock_df["成交额"] > 5e8)
]

print("大资金流入候选股：")
print(candidates_1[["代码", "名称", "最新价", "涨跌幅", "换手率", "量比", "成交额"]])

大资金流入候选股：
          代码       名称    最新价    涨跌幅    换手率     量比           成交额
6     301382      蜂助手  47.64  20.00  27.27   2.11  1.551669e+09
8     300148     天舟文化   5.10  20.00  27.28   3.05  1.049429e+09
10    301139     元道通信  31.82  19.98  28.32   3.08  6.111752e+08
12    300159     新研股份   2.89  19.92  18.25   2.54  7.484901e+08
13    301202     朗威股份  58.10  18.52  63.54   2.37  1.298787e+09
15    301336     趣睡科技  88.01  15.80  32.40   2.05  6.524976e+08
16    300873     海晨股份  22.10  14.98  20.88   6.13  7.636306e+08
21    688327  云从科技-UW  19.45  13.08  27.33   2.18  3.999460e+09
22    300475     香农芯创  36.99  13.05  25.66   3.23  3.996801e+09
31    300310     宜通世纪   8.96  10.75  27.22   2.58  1.642886e+09
54      2522     浙江众成   6.03  10.04  17.89   6.81  9.359171e+08
77      2401     中远海科  18.69  10.01  15.84   4.71  1.079470e+09
94      2290     禾盛新材  26.52  10.00  19.05   2.60  1.207717e+09
110      573     粤宏远Ａ   4.44   9.90  19.97   2.62  5.468631e+08
127   300411     金盾股份  15.07  

In [98]:
import pandas as pd

# 将 '代码' 列转换为字符串类型
candidates_1["代码"] = candidates_1["代码"].astype(str)
candidates_2["代码"] = candidates["代码"].astype(str)

# 通过 '代码' 列进行合并
common_df = pd.merge(candidates_1, candidates, on="代码", how="inner")

# 打印重合的行
print("重合的股票：")
print(common_df[["代码", "名称_x", "最新价_x", "涨跌幅_x", "换手率_x", "量比_x", "成交额_x"]])

# 打印重合的股票代码
common_codes = common_df["代码"].unique()
print(f"\n重合的股票代码：{common_codes}")

# 如果没有重合
if common_df.empty:
    print("\n两个 DataFrame 之间没有重合的股票。")


重合的股票：
       代码 名称_x  最新价_x  涨跌幅_x  换手率_x  量比_x        成交额_x
0  301013  利和兴  14.46   5.55  20.67  2.29  556681923.0

重合的股票代码：['301013']


C:\Users\charl\AppData\Local\Temp\ipykernel_68924\1444991662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates_1["代码"] = candidates_1["代码"].astype(str)
C:\Users\charl\AppData\Local\Temp\ipykernel_68924\1444991662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates_2["代码"] = candidates["代码"].astype(str)


In [99]:
import akshare as ak

# 获取指数日线数据（示例：上证指数 sh000001）
symbol = "sh000001"
index_daily = ak.stock_zh_index_daily(symbol=symbol)

# 计算每日涨跌幅（收盘价相对于前一日的涨跌百分比）
index_daily['涨跌幅'] = index_daily['close'].pct_change() * 100

# 获取最新交易日尾盘（收盘）涨跌幅
latest_change = index_daily['涨跌幅'].iloc[-1]
print(f"最新交易日收盘涨跌幅: {latest_change:.2f}%")

最新交易日收盘涨跌幅: 1.02%


In [109]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta

# 定义主要指数代码（格式：akshare 兼容的代码）
index_list = {
    "上证指数": "sh000001",
    "沪深300": "sh000300",
    "深证成指": "sz399001",
    "创业板指": "sz399006",
    "科创50": "sh000688"
}

def get_index_last_30min_change(symbol: str) -> float:
    """获取指数最后30分钟涨幅（基于分钟数据）"""
    try:
        # 获取分钟级数据（周期为1分钟）
        minute_data = ak.stock_board_concept_hist_min_em(symbol="上证指数", period="30")

        minute_data['日期时间'] = pd.to_datetime(minute_data['日期时间'])
        
        # 筛选交易日的最后30分钟（14:30-15:00）
        last_date = minute_data['日期时间'].max().date()  # 最新日期
        mask = (minute_data['日期时间'] >= pd.Timestamp(last_date).replace(hour=14, minute=30)) & \
               (minute_data['日期时间'] <= pd.Timestamp(last_date).replace(hour=15, minute=0))
        last_30min_data = minute_data[mask]
        
        if last_30min_data.empty:
            return 0.0
        
        # 计算涨幅：起始价（14:30的收盘价）到结束价（15:00的收盘价）
        start_price = last_30min_data.iloc[0]['收盘']
        end_price = last_30min_data.iloc[-1]['收盘']
        return (end_price - start_price) / start_price * 100
    except Exception as e:
        print(f"获取 {symbol} 分钟数据失败: {e}")
        return 0.0

# 获取各指数收盘价及最后30分钟涨幅
results = []
for index_name, symbol in index_list.items():
    # 获取日线数据（收盘价）
    daily_data = ak.stock_zh_index_daily(symbol=symbol)
    latest_close = daily_data['close'].iloc[-1]  # 最新收盘价
    
    # 获取最后30分钟涨幅
    last_30min_change = get_index_last_30min_change(symbol)
    
    results.append({
        "指数名称": index_name,
        "收盘价": round(latest_close, 2),
        "最后30分钟涨幅 (%)": round(last_30min_change, 2)
    })

# 转换为DataFrame输出
result_df = pd.DataFrame(results)
print("\nA股主要指数收盘价及尾盘涨幅:")
print(result_df)

获取 sh000001 分钟数据失败: index 0 is out of bounds for axis 0 with size 0
获取 sh000300 分钟数据失败: index 0 is out of bounds for axis 0 with size 0
获取 sz399001 分钟数据失败: index 0 is out of bounds for axis 0 with size 0
获取 sz399006 分钟数据失败: index 0 is out of bounds for axis 0 with size 0
获取 sh000688 分钟数据失败: index 0 is out of bounds for axis 0 with size 0

A股主要指数收盘价及尾盘涨幅:
    指数名称       收盘价  最后30分钟涨幅 (%)
0   上证指数   3380.21           0.0
1  沪深300   3959.94           0.0
2   深证成指  10955.65           0.0
3   创业板指   2268.22           0.0
4   科创50   1126.82           0.0


In [110]:
import akshare as ak

index_list = {
    "上证指数": "sh000001",
    "沪深300": "sh000300",
    "深证成指": "sz399001",
    "创业板指": "sz399006",
    "科创50": "sh000688"
}

for index_name, index_code in index_list.items():
    try:
        # 获取30分钟数据，period设置为'30'
        df = ak.stock_zh_index_minute(symbol=index_code, period='30')
        print(f"{index_name} ({index_code}) 的30分钟数据：")
        print(df.head())  # 打印前几行数据
    except Exception as e:
        print(f"获取 {index_name} ({index_code}) 数据失败，错误信息：{e}")

获取 上证指数 (sh000001) 数据失败，错误信息：module 'akshare' has no attribute 'stock_zh_index_minute'
获取 沪深300 (sh000300) 数据失败，错误信息：module 'akshare' has no attribute 'stock_zh_index_minute'
获取 深证成指 (sz399001) 数据失败，错误信息：module 'akshare' has no attribute 'stock_zh_index_minute'
获取 创业板指 (sz399006) 数据失败，错误信息：module 'akshare' has no attribute 'stock_zh_index_minute'
获取 科创50 (sh000688) 数据失败，错误信息：module 'akshare' has no attribute 'stock_zh_index_minute'


In [1]:
import akshare as ak

# 定义指数列表
index_list = {
    "上证指数": "000001",
    "沪深300": "000300",
    "深证成指": "399001",
    "创业板指": "399006",
    "科创50": "000688"
}

# 遍历指数列表，获取数据
for index_name, index_code in index_list.items():
    try:
        # 使用 stock_zh_index_hist_min_em 方法获取指数数据
        df = ak.index_zh_a_hist(
            symbol=index_code, 
            period='30',  # 30分钟周期
            start_date="2025-02-26 09:30:00",  # 自定义开始日期
            end_date="2025-02-26 15:00:00"     # 自定义结束日期
        )
        print(f"{index_name} ({index_code}) 的30分钟数据：")
        print(df)
    except Exception as e:
        print(f"获取 {index_name} ({index_code}) 数据失败，错误信息：{e}")

获取 上证指数 (000001) 数据失败，错误信息：'30'
获取 沪深300 (000300) 数据失败，错误信息：'30'
获取 深证成指 (399001) 数据失败，错误信息：'30'
获取 创业板指 (399006) 数据失败，错误信息：'30'
获取 科创50 (000688) 数据失败，错误信息：'30'
